In [1]:
# Importar bibliotecas necesarias
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

# Descripción:

# Este código realiza la generación de un modelo de clasificación utilizando un conjunto de datos proporcionado en el 
# archivo CSV "datos_tarea_voluntaria.csv". El objetivo es predecir la variable "cell_line" (línea celular) basándose 
# en el nivel de expresión de ciertas proteínas.

# Justificación de la selección:

# Este enfoque se seleccionó porque el RandomForest es un algoritmo robusto y versátil que puede manejar tanto variables 
# numéricas como categóricas. La búsqueda de hiperparámetros a través de GridSearchCV permite encontrar la combinación 
# óptima para maximizar la exactitud del modelo. La toma de una muestra aleatoria del conjunto de datos facilita el 
# desarrollo y la experimentación sin comprometer la representatividad del modelo.

# Alternativas a considerar:

# Durante el desarrollo del modelo, se exploraron diferentes técnicas y modelos para comparar su rendimiento con el 
# RandomForest seleccionado. Algunas de las alternativas probadas incluyen:
# - KNN: Se probó el clasificador KNN para evaluar la eficacia de un enfoque basado en vecinos más cercanos
# - SVM: Se evaluó para determinar si podría superar las limitaciones de linealidad.
# - Gradient Boosting: Se exploró el modelo, conocido por su capacidad para mejorar la precisión mediante la combinación de varios modelos más simples

# Nota:

# Me tardó en ejecutar el fit aproximadamente 3 minutos, los resultados obtenidos son:
# - Mejores hiperparámetros: {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100}
# - Test Accuracy: 0.9141579731743666

# Cargar los datos
df = pd.read_csv('datos_tarea_voluntaria.csv')

# Ajustamos la fracción para reducir el tiempo de ejecución
df_sample = df.sample(frac=0.1, random_state=42)

# Dividir el conjunto de datos en entrenamiento y prueba
X = df_sample.drop('cell_line', axis=1)
y = df_sample['cell_line']

In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definir variables para preprocesamiento
numeric_features = df.columns.tolist()[:-1]
categorical_features = []  # Si hay columnas categóricas, agréguelas aquí

In [3]:
# Crear un pipeline de preprocesamiento
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Incluir el clasificador en el pipeline
model = RandomForestClassifier(random_state=42)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

param_grid = {
    'classifier__n_estimators': [50, 100],
    'classifier__max_depth': [5, 10],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2]
}

In [4]:
# Crear un objeto GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

# Entrenar el grid en el conjunto de datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo y sus hiperparámetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros:", best_params)

Mejores hiperparámetros: {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100}


In [5]:
# Utilizar el mejor modelo para hacer predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)

# Evaluar la accuracy en el conjunto de test
accuracy_test = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy_test)

Test Accuracy: 0.9141579731743666


In [6]:
# Guardar el mejor modelo en formato binario
import joblib
joblib.dump(best_model, 'mejor_modelo.pkl')

['mejor_modelo.pkl']